In [1]:
import torch
import torch.nn as nn
import torch.tensor as tensor
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.5.1+cpu'

In [36]:
BATCH_SIZE=1
EPOCHS=2

In [37]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [38]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
#         # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
#         self.conv1 = nn.Conv2d(1, 10, 5) # 输入通道数1，输出通道数10，核的大小5
#         self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
#         # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
#         self.fc1 = nn.Linear(20*10*10, 500) # 输入通道数是2000，输出通道数是500
#         self.fc2 = nn.Linear(500, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
        in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10） 拼接成一行
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

In [6]:
model = ConvNet()
optimizer = optim.Adam(model.parameters())

ValueError: optimizer got an empty parameter list

In [7]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
#         data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
#             data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [45]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, epoch)
    test(model, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.304911
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.167282
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.131665

Test set: Average loss: 0.0854, Accuracy: 9758/10000 (98%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.081635
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.065977
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.049058

Test set: Average loss: 0.0534, Accuracy: 9830/10000 (98%)



In [52]:
MNIST_CNN_PARAM = '../modelparam/MNIST.pt'

In [47]:
torch.save(model.state_dict(), MNIST_CNN_PARAM)

In [48]:
class ConvNet_Left(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
        self.conv1 = nn.Conv2d(1, 10, 5) # 输入通道数1，输出通道数10，核的大小5
        self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
        # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
#         self.fc1 = nn.Linear(20*10*10, 500) # 输入通道数是2000，输出通道数是500
#         self.fc2 = nn.Linear(500, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
        in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
#         out = self.fc1(out) # batch*2000 -> batch*500
#         out = F.relu(out) # batch*500
#         out = self.fc2(out) # batch*500 -> batch*10
#         out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

In [49]:
class ConvNet_Right(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
#         self.conv1 = nn.Conv2d(1, 10, 5) # 输入通道数1，输出通道数10，核的大小5
#         self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
        # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
        self.fc1 = nn.Linear(20*10*10, 500) # 输入通道数是2000，输出通道数是500
        self.fc2 = nn.Linear(500, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
#         in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
#         out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
#         out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
#         out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
#         out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
#         out = F.relu(out) # batch*20*10*10
#         out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
        out = x
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

In [50]:
model_left = ConvNet_Left()
model_right = ConvNet_Right()

In [51]:
model_left.load_state_dict(torch.load(MNIST_CNN_PARAM), strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])

In [52]:
model_right.load_state_dict(torch.load(MNIST_CNN_PARAM), strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias'])

In [53]:
model_left.eval()
model_right.eval()

ConvNet_Right(
  (fc1): Linear(in_features=2000, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [53]:
dataiter = iter(test_loader)
input_data, label = dataiter.next()

In [55]:
with torch.no_grad():
    output = model_left(input_data)
    print(output)

tensor([[0.0000, 0.0785, 0.3710,  ..., 0.9503, 0.9624, 0.8688],
        [0.0000, 0.2101, 0.4535,  ..., 0.1309, 0.2444, 0.1742],
        [0.0000, 0.1116, 0.0000,  ..., 0.2882, 0.2697, 0.4737],
        ...,
        [0.0000, 0.1960, 0.5782,  ..., 1.1049, 0.7386, 0.1611],
        [0.0011, 0.0000, 0.0334,  ..., 0.1883, 0.2554, 0.4161],
        [0.0000, 0.0000, 0.0000,  ..., 0.3858, 0.3701, 0.3795]])


In [56]:
with torch.no_grad():
    output = model_right(output)

In [57]:
pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
print(len(pred))
correct = pred.eq(label.view_as(pred)).sum().item()

512


In [58]:
print(correct)

505


In [42]:
class ConvNet_MNIST(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
       
        self.layer_list = nn.ModuleList([
            nn.Conv2d(1,10, 5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(10, 20, 3),
            nn.ReLU(),
            nn.Linear(20*10*10, 500),
            nn.ReLU(),
            nn.Linear(500, 10),
            nn.LogSoftmax(dim=1)
        ])
        self.layer_size = len(self.layer_list)
    def forward(self,x, start=0, end=9):
#         in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
#         out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
#         out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
#         out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
#         out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
#         out = F.relu(out) # batch*20*10*10
#         out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10） 拼接成一行
#         out = self.fc1(out) # batch*2000 -> batch*500
#         out = F.relu(out) # batch*500
#         out = self.fc2(out) # batch*500 -> batch*10
#         out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        in_size = x.size(0)
        out = x
        for i in range(start, end):
#             print(i)
            out = self.layer_list[i](out)
            if i == 4:
                out = out.view(in_size, -1)
        return out

In [43]:
dataiter = iter(test_loader)
input_data, label = dataiter.next()
# input_data = input_data[1]
print(input_data.size())

torch.Size([1, 1, 28, 28])


In [44]:
mnist_model_part_1 = ConvNet_MNIST()
mnist_model_part_2 = ConvNet_MNIST()
optimizer = optim.Adam(mnist_model_part_1.parameters())
mnist_model_part_1.load_state_dict(torch.load(MNIST_CNN_PARAM))
mnist_model_part_2.load_state_dict(torch.load(MNIST_CNN_PARAM))

<All keys matched successfully>

In [45]:
# mnist_model = ConvNet_MNIST()
# optimizer = optim.Adam(mnist_model.parameters())
# for epoch in range(1, EPOCHS):
#     train(mnist_model, train_loader, optimizer, epoch)
#     test(mnist_model, test_loader)

In [46]:
MNIST_CNN_PARAM = '../modelparam/MNIST.pt'
# torch.save(mnist_model.state_dict(), MNIST_CNN_PARAM)

In [54]:
with torch.no_grad():
    output = mnist_model_part_1(input_data, 0, 5)


In [55]:
with torch.no_grad():
    output = mnist_model_part_2(output, 5, 9)
pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
print("predict number:",pred)
print("label:", label[0])
print(len(pred))
correct = pred.eq(label.view_as(pred)).sum().item()
print(correct)

predict number: tensor([[8]])
label: tensor(8)
1
1


In [56]:
print(output.size())

torch.Size([1, 10])
